In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from nltk.corpus import stopwords
from itertools import combinations
import string
from collections import Counter, defaultdict
import re

In [2]:
path_connected_data = "C:\\Users\\tykun\\OneDrive\\Documents\\SchoolDocs\\VSCodeProjects\\connectedData\\"
path_temp_data = "C:\\Users\\tykun\\OneDrive\\Documents\\SchoolDocs\\VSCodeProjects\\connectedData\\temporaryData\\"
year = "2009"

full_path = f"{path_connected_data}{year}_split_positions.csv"
dean_path = f"{path_temp_data}{year}_deans.csv"
grouped_subinstitution_path = f"{path_temp_data}{year}_grouped.csv"

pd.options.display.max_rows = 400

In [3]:
POSITION_BANK = ["President", "Chancellor", "Provost", "Director", "Dean", "Controller", "Trustee", "Member", "Regent", "Chairman", "Overseer", "Assistant", "Librarian", "Secretary", "Chaplain", "Minister", "Treasurer", "Senior Counsel", "General Counsel", "Legal Counsel", "University Counsel", "College Counsel", "Special Counsel", "Corporation Counsel", "Corporate Counsel", "Officer", "Chief", "Professor", "Commissioner", "Fellow", "Chairperson", "Manager", "Clergy", "Coordinator", "Auditor", "Governor", "Representative", "Stockbroker", "Advisor", "Commandant", "Rector", "Attorney", "Curator", "Clerk", "Department Head", "Pastor", "Head", "Comptroller", "Deputy", "Inspector General", "Instructor", "Registrar", "Ombuds", "Administrator", "Liaison", "Administrative Associate", "Webmaster", "Specialist", "University Planner", "Architect"]
#for classifying position as dean, administration
DEAN_WORDS = ["summer", "student", "faculty", "academic service", "academics", "academic program", "admissions", "admission", "enrollment", "student life", "housing", "academic support", "advising", "enrollment management", 
                       "student relations", "academic computing", "academic records", "student service", "student affairs", "student development", "registrar", "financial aid", "media service", "library service", "university librar",
                       "internation affair", "special program", "continuing education", "external relation", "development", "services"]

#for subinstituion
ADMINISTRATION_WORDS = ["academic service", "academics", "academic program", "admissions", "admission", "enrollment service", "student life", "housing", "academic support", "advising", "enrollment management", 
                       "student relations", "academic computing", "academic records", "student service", "student affairs", "student development", "registrar", "financial aid", "media service", "library service", "university librar"]

In [4]:
#define keywords
keywords = {
    "Arts and Sciences": ["arts and science", "letters and science", "arts, sciences, and letters", "arts, sciences", "arts & sciences", "letters, sciences", "arts and letters", "arts & letters"],
    "Engineering": ["engineering", "mines", "aerospace", "aeronautics", "polytechnic studies", "aviation", "applied science", "computational science"],
    
    "Computer Science": ["computer science", "computer,", "software engineering", "and computer", "computing"],
    "Data Science": ["data science", "artificial intelligence", "machine learning"],
    "Technology": ["technology"],
    "Information": ["information science", "information management", "information systems", "information stud", "library science", "and information", "library stud", "informatics", "school of information", 
                    "information and", "information school"],
    "Archictecure": ["architecture", "architectural", "urban design", "urban planning", "environmental design", "environment and design", "design, construction", "school of construction"],
    

    "Behavioral Science": ["psychology", "psychological", "human science", "human ecology", "behavioral", "human and", "childhood stud"],
    "Social Sciences": ["social science", "social and", "social ecology"],
    "Natural Sciences": ['natural science', "mathematics", "chemistry", "biology", "mathematical", "physical science", "life science", "biological science", 
                        "biochemistry", "natural and", "basic", "college of science", "school of science", ", science, and", ", science and", "physics", "college of the sciences"],
    "Environmental and Earth Science": ["environmental science", "forestry", "natural resource", "college of forest", 
                              "school of forest", "school of environment ", "environmental and", "geology", "oceanography", "earth science", "geophysics", "seismology", "volcanology", "earth and" 
                              "paleontology", "mineral science", "geoscience", "earth science", "earth studies", "atmospheric", "college of the environment", "school of ecology", "school of environment"],
    "Marine Science": ["marine science", "marine stud", "ocean science", "oceanography", "oceanographic", "ocean and"],

    
    "Business": ["business", "commerce", "mba", "college of economics", "and economics", "finance", "entrepreneurship", "school of management", "and management", "accounting", "agribusiness", "economic,", "accountancy", "schools of management", "college of management"],
    "Hospitality": ["hospitality", "hotel administration", "hotel management", "restaurant management"],
    "Education": ["college of education", "school of education", "teaching", "teach", "curriculum development", "human development", "and education", ", education and"],
    "Criminal Justice": ["criminal justice", "criminology", "criminal", "forensics"],


    "Law": ["law", "legal studies"],
    "Policy and Affairs": ["public policy", "international policy", "public affairs", "international affairs", "environmental affairs", "urban affairs", "metropolitan affairs", "international service", "leadership studies", 
                           "diplomacy", "diplomat", "international studies", "public administration", "urban policy", "global studies", "international relations", "social policy", "peace studies", "global management", "policy studies",
                           "policy, planning", "school of leadership", "policy science"],
    

    "Agriculture": ["agriculture", "agricultural", "agribusiness"],
    "Family and Consumer Science": ["family and", "home economics", "consumer science", "family science", "home", "family studies"],
    "Culinary": ["culinary", "cooking", "gastronomy"],


    "Journalism": ["journalism"],
    "Communication": ["communication"],
    "Liberal Arts": ["liberal art", "liberal studies", "interdisciplinary studies", "general studies", "liberal and", "individualized study"],
    "Humanities": ["humanities", "history", "historical", "philosophy", "literature", "political"],
    "Fine Arts": ["fine arts", "music", "theater", "creative", "theatre", "film", "performing art", "the arts", "design art", "dramatic arts", "dance",
                   "visual arts", "school of drama", "recording arts", "ceramics", "fine and applied art", "school of design", "art and design", "fine", 
                   "cinema", "art ", "art, ", "studio art", "media art", "arts and performance", "drama", "arts and media", "textiles", "applied arts"],
    "Language and Culture": ["languages", "lingusitics", "cultures", "translational studies", "intercultural studies", "ethnic studies", "hebraic studies", "cultural stud", "asian studies", "culture ", " hawaiian knowledge", 
                             "jewish", "asian and pacific studies"],


    "Nursing": ["nursing"],
    "Physical Therapy and Nutrition": ["physical therapy", "nutrition", "physiotherapy", "nutritional therapy", "rehab", "human performance"],
    "Medical": ["medical", "pharmacy", "surgeons", "school of medicine", "college of medicine", "podiatric medicine", "osteopathic medicine", "human medicine"],
    "Pharmacy": ["pharmacy", "pharmacology"],
    "Dentistry" : ["dentistry", "dental", "denistry"],
    "Optometry" : ["optometry", "optics", "optical science"],
    "Veterinary" : ['veterinary'],
    "Audiology": ["audiology"],
    "Gerontology": ["gerontology"],
    "Pathology": ["pathology"],
    "Health Science": ["health science", "public health", "health profession", "college of health", "school of health", "biomedical science", "and health", "allied health", "physical education",
                        "health and", "health studies", "genome", "occupational therapy"],


    "Religious Studies": ["religious", "theology", "divinity", "biblical", "theological", "religion", "seminary", "religous", "christ college", "talmudic", "torah", "quran", "bible"],
    "Social Work": ["social work", "social welfare", "community service", "human service", "social service", "public service", "social administration"],
    

    "Graduate": ["graduate school", "graduate studies", "graduate college", "college of graduate", "school of graduate",  "advancing", "advanced study", "graduate division", "graduate and professional studies", "doctoral", "graduate study", "graduate "],
    "Continued Studies": ["extended studies", "extended education", "continuing education", "continued education", "continuing studies", "lifelong learning", "professional programs", "extended learning", "adult learning",
                           "adult education", "online learning", "adult student", "lifelong education", "continuing and", "university extension", "distance learning", "continuous education", "extended university"],
    "Professional Studies": ["professional studies", "professional development", "professional education", "school of professions", "professional campus"],
    "Administration": ["academic service", "academics", "academic program", "admissions", "admission", "enrollment service", "student life", "housing", "academic support", "advising", "enrollment management", 
                       "student relations", "academic computing", "academic records", "student service", "student affairs", "student development", "registrar", "financial aid", "media service", "library service", 
                       "university librar", "information service", "information access", "library access", "residence", "academic affairs", "library affairs", "academic succes", "first year student",
                       "undergraduate studies", "undergraduate education"],
    "Honor College": ["honor college", "honors college", "scholar program"],

    "Satellite Campus": ["campus"],
    "Foundation": ["foundation"],
    "Boards": ["board of", "regents", "trustees", "council of"],
    "Hospital": ["hospital"],
    "University College": ["university college"],
    "Error": [],
    "Dean of the College": ["dean of the college", "dean, undergraduate college", "dean of the university"]
}

In [5]:
#slightly altered from the vp file
VP_KEYWORDS = {
    # "Head VP": ["head vice president"],
    # "SubInstitution": ["arts and science", "the college", "the arts", "interdisciplinary studies", "life sciences", "agriculture", "agricultural", "liberal arts", "engineering", "law center"],
    "Generic": [],
    "Foundation": ["foundation"],


    "Administration": ["vice president, administration", "and administration", "administrative", "audit and compliance", "legal affairs", "legal services", "institutional affairs", "university affairs", "vice president, operations", 
                       "employment service", "vice president, quality", "management service", "university compliance" "university operation", "institutional services",
                       "of the corporation", "special program", "university governance", "materials management", "executive affair"," university operation", "vice president. administration", "administration",
                       "resources"],
    "Board VP": ["of the board", "board of", "board operation"],
    "Campus Operations": ["sustainability", "facilities", "campus operations", "infrastructure", "environmental", "security", "campus police", "emergency management", "safety", 
                          "campus services", "university services", "facility operation", "university architect", "land, buildings and real estate", "auxiliary", "capital construction",
                          "property management", "plant operations", "construction", "campus residence", "physical resources", "university compliance", "physical plant", "campus involvement"],
    "Health Science": ["health science", "public health", "health profession", "college of health", "school of health", "biomedical science", "and health", "allied health", "physical education",
                        "health and", "health studies", "genome"],
    "Medical": ["medical", "pharmacy", "surgeons", "school of medicine", "college of medicine", "podiatric medicine", "osteopathic medicine"],



    "Academic Affairs": ["undergraduate education", "curriculum", "provost" "undergraduate studies", "graduate studies", "instruction", "academic program", "academic development",
                         "education affair", "undergraduate affair", "academie affairs", "educational and", "teacher education", "experiential education", "academic affair", "academic administration",
                         "undergraduate initiative", "cooperative education", "school program", "interdisciplinary education", "educational outreach", "educational,", "undergraduate studies", "educational affair",
                         "academic service", "academic afair", "educational resource", "academic enrichment", "academic resource", "university studies", "vice president, academics", "academic operation",
                         "educational development", "academic issues", "academic planning", "academic department"],
    "Faculty Affairs": ["faculty", "staff relations", "faculties"],
    "Research": ["innovation", "research", "grants", "publication", "biotechnology", "scientific", "sponsored programs"],
    

    "Student Affairs": ["student", "student life", "student service", "support", "wellbeing", "well-being", "campus life", "housing", "counseling", "student affairs", "student initiative", "student service", 
                        "student success", "student activities", "student development", "students", "university life", "student learning", "residence life", "student and community",
                        "career service", "student judicial service", "advisement", "student access", "study abroad", "dining program", "academic life", "freshmen", "freshman"],
    "Enrollment": ["enrollment", "financial aid", "retention", "registrar", "admission", "enrolment"],
    

    "Information and Technology": ["technology", "information", "librar", "data management", "computer", "computing", "network service", "internet service", "electronic marketing", "converging technologies",
                    "technical service", "vice president, systems", "enterprise system", "records", "system administration"],


    "DEI": ["diversity", "equity", "inclusion", "affirmative action", "dei", "multicultural affairs", "equal opportunity", "minority affair", "intercultural affair"],
    "External Relations": [ "external", "university relation", "college relation", "public relation", "communications", "public affair", "media", "outreach", "partnership", "global",
                            "international", "study abroad", "federal relations", "public events", "institutional relation",
                           "government", "state relations", "community engagement", "community affair", "civic engagement", "community relation", "corporate relation", 
                           "foundation relation", "legislative", "policy affairs", "defense relation", "constituent relations", "regional affair", "system relation", "regional operation",
                           "federal program", "agency relation", "regional engagement", "communication", "news services", "federal affair", "commonwealth relation", "state affair", "institute relation", "community services",
                           "intercampus", "urban affairs", "public service", "school relation", "public policy", "district of columbia affair", "d.c. affair", "vice president, engagement", "vice president, policy", "institute affair",
                           "community development"],


    "Planning": ["planning", "assessment", "evaluation", "planner", "special projects", "strategy and measurement", "institutional effectiveness", "presidential initiative", "capital improvement", "strategy and policy",
                 "university projects", "institutional analysis", "commercialization strategies", "procurement", "strategic affair", "university initiative", "program development", "strategic initiative", "strategic development",
                 "strategic and", "project", "program"],
    "Finance": ["budget", "finance", "financial", "accounting", "marketing", "endowment",  "investment", "business", "fiscal", "and resource management", "institutional resources", "treasury management", "university resources", "audit", ", resource management",
                "commerce", "asset management"],
    "Advancement": ["fundraise", "fundraising", "alumni", "donation", "parent relation", "annual giving", "special events", "principal gifts", "major gift", "campaigns", 
                    "planned giving", "leadership gifts", "gift program", "trusts and estate", "giving", "philanthropy", "constituent program", "institutional advancement", "vice president, advancement", "vice president, development",
                    "university advancement", "university development", "vice president. development", "vice president. advancement", "economic advancement", "economic development",
                    "college advancement", "institute advancement", "and development", "and advancement", "resource development", "institutional advancement", "sponsored funds", "college development", "wide advancement", "institution advancement",
                    "regional development", "development", "advancement"],
    "Human Resources": ["hr", "human resource", "personnel", "recruitment", "employee relations", "union", "collective bargaining", "labor", "human relations", "human capital", "personnel", "human and", "professional development", "human service", "hiring",
                        "workforce"],
    

    "Religion": ["ministry", "mission", "ministries", "church", "religious", "religion", "spiritual", "christian development"],
    "Satellite Campus": ["satellite campus"],
    "Athletics": ["athletic", "sports"],
    # "Agriculture": ["agriculture", "agricultural"],
    # "Continued Studies": ["extended studies", "extended education", "continuing education", "continued education", "continuing studies", "lifelong learning", "professional programs", "extended learning",
    #                        "adult education", "online learning", "adult student", "lifelong education", "continuing and", "university extension", "extended program", "distance learning", "continuous education",
    #                        "online program", "online school", "online college", "online university", "distance education", "extended service"],
    "Graduate": ["graduate program", "graduate school", "graduate and", "advanced study", "graduate education", "graduate studies", "graduate "],
    "Subinstitution Match":  ["Arts and Sciences", "Engineering", "Applied Science", "Computer Science", "Data Science", "Technology", "Information", "Architecture", "Behavioral Science", "Social Sciences", "Natural Sciences", "Environmental and Earth Science", "Marine Science", "Business", "Hospitality", "Criminal Justice", "Law", "Policy and Affairs", "Agriculture", "Family and Consumer Science", "Culinary", "Journalism", "Communication", "Liberal Arts", "Humanities", "Fine Arts", "Language and Culture", "Nursing", "Physical Therapy and Nutrition", "Pharmacy", "Dentistry", "Optometry", "Veterinary", "Audiology", "Gerontology", "Pathology", "Social Work", "Continued Studies", "Professional Studies", "Honor College", "Hospital"],
    "Other": []
}

In [6]:
grouped = pd.read_csv(grouped_subinstitution_path)
satellites = []
for index, row in grouped.iterrows():
    sub = row["SubInstitution"]
    category = row["Category"]
    if category == "Satellite Campus":
        satellites.append(sub)
VP_KEYWORDS["Satellite Campus"] = satellites
print(satellites)

['Downtown Phoenix Campus', 'Polytechnic Campus', 'Tempe Campus', 'Arizona State University at the West Campus', 'Metropolitan College', 'BGSU Firelands', 'California State University, San Bernardino in Palm Desert', 'Carnegie Mellon University in Qatar', 'Carnegie Mellon West', 'College at Florham Campus', 'Metropolitan Campus', 'Mount Vernon Campus', 'College of Virginia Beach', 'Indiana University East [IU East]', 'Indiana University Kokomo [IUK]', 'Indiana University Northwest [IU Northwest]', 'Armstrong Campus', 'Punxsutawney Campus', 'Brooklyn Campus', 'CW Post Campus', 'Long Island University at Riverhead', 'Hamilton', 'Middletown', 'West Plains Campus', 'Mountain Grove Research Campus', 'Alamogordo Campus', 'Carlsbad Campus', 'Dona Ana Campus', 'Grants Campus', 'Northwestern University in Qatar', 'OSU-Cascades Campus', 'Pennsylvania State University - Abington College', 'Pennsylvania State University - Altoona College', 'Pennsylvania State University - Berks College', 'Pennsylv

In [7]:
admin_types = ["Human Resources", "Advancement", "Finance", "Planning", "Academic Affairs", "Faculty Affairs", "External Relations", "Campus Operations", "Research", "Board VP", "Administration", "Foundation", "Generic", "Enrollment" ]

In [8]:
#Extract the names of all the institutions for validation
def extract_institutions(df):
    institution_list = []
    for index, row in df.iterrows():
        if row["Institution"] not in institution_list:
            institution_list.append(row["Institution"])
    return institution_list


def extract_first_member(df):
    first_member_df = []
    previous_institution = None
    for index, row in df.iterrows():
        current_institution = row["Institution"]
        if previous_institution is None or current_institution != previous_institution:
            first_member_df.append(row)
        previous_institution = current_institution
    return pd.DataFrame(first_member_df)

def find_missing_institutions(institutions, df):
    lowercase_df = df["Institution"].str.lower()
    missing_institutions = []
    for institution in institutions:
        if institution.lower() not in lowercase_df.values:
            missing_institutions.append(institution)
    return missing_institutions

def rename_previous_value(df):
    df.rename(columns={'Previous_Value': 'FixedPosition'}, inplace = True)
    df["FixedPosition"] = ""
    return pd.DataFrame(df)

def count_universities(full_df):
    institutions = full_df["Institution"].unique()
    return list(institutions)

In [9]:
#Deans
#more to add here, but probably better to finish subinstitutions problem before doing this.
def extract_deans(df):
    dean_df = []
    for index, row in df.iterrows():
        if ('dean' in row["Position"].lower()) and "assistant to the dean" not in row["Position"].lower():
            dean_df.append(row)
    dean_df = pd.DataFrame(dean_df)
    return dean_df

#remove parentheses at end of string *used for Dean (interim) or Dean (acting)
def remove_parentheses(string):
    index = string.find('(')
    if index != -1:
        string = string[:index]
    return string.title()

def assign_position_grouping(position):
    classification = ""
    for key, value in VP_KEYWORDS.items():
        value_upper = [s.title() for s in value]
        keyword_appears = any(s in position for s in value_upper)
        dean_comma = position.rfind("Dean,") + 5
        if key.title() == position[dean_comma:]:
            classification += key + ", "
        elif keyword_appears:
            classification += key + ", "
    return classification.strip(", ")

def assign_position_grouping_subinst(position):
    classification = ""
    for key, value in keywords.items():
        value_upper = [s.title() for s in value]
        keyword_appears = any(s in position for s in value_upper)
        dean_comma = position.rfind("Dean,") + 5
        if key.title() == position[dean_comma:]:
            classification += key + ", "
        elif keyword_appears:
            classification += key + ", "
    return classification.strip(", ")


def assign_deans_subinstitution(deans_df, subinstitution_df):
    deans_df["FixedPosition"] = ""
    subinstitution_dict = subinstitution_df.set_index(['Institution', 'SubInstitution'])['Category'].to_dict()
    for index, row in deans_df.iterrows():
        position = row["Position"].title()
        altered_position = remove_parentheses(position).strip()
        subinstitution = row["SubInstitution"]
        dean_index = altered_position.rfind("Dean") + 4
        assigned = False
        #no text after "dean" and subinstitution exists
        if dean_index == len(altered_position) and type(subinstitution) != float:
            key = (row["Institution"], row["SubInstitution"])
            deans_df.at[index, "FixedPosition"] = subinstitution_dict[key]
        #no text after "dean" and subinstitution does not exist:
        if dean_index == len(altered_position) and type(subinstitution) == float:
            deans_df.at[index, "FixedPosition"] = "Dean Of The College"
        #text after "dean" and subinstitution exists
        elif dean_index != len(altered_position) and type(subinstitution) != float:
            #use subinst classification for the position
            classification = assign_position_grouping_subinst(position)
            deans_df.at[index, "FixedPosition"] = classification
            # print(position, "    ", classification)
        elif dean_index != len(altered_position) and type(subinstitution) == float:
            classification = assign_position_grouping(position)
            deans_df.at[index, "FixedPosition"] = classification
            assigned = True
    return deans_df

def assign_remaining(deans_df, subinstitution_df):
    subinstitution_dict = subinstitution_df.set_index(['Institution', 'SubInstitution'])['Category'].to_dict()
    for index, row in deans_df.iterrows():
        fixed = row["FixedPosition"]
        subinstitution = row["SubInstitution"]
        position = row["Position"].title()
        altered_position = remove_parentheses(position).strip()
        dean_index= altered_position.rfind("Dean") + 4
        if fixed == "":
            classification = assign_position_grouping_subinst(position)
            # print(classification)
            deans_df.at[index, "FixedPosition"] = classification
        if dean_index == len(position) and fixed == "":  #and type(subinstitution) == float:
            deans_df.at[index, "FixedPosition"] = "Dean of the College"
        if "Dean Of The College" in position:
            deans_df.at[index, "FixedPosition"] = "Dean of the College"
    return deans_df
            

def assign_none(deans_df):
    for index, row in deans_df.iterrows():
        fixed = row["FixedPosition"]
        if fixed == "":
            deans_df.at[index, "FixedPosition"] = "Administration"
    return deans_df


def assign_administration(deans_df):
    for index, row in deans_df.iterrows():
        fixed = row["FixedPosition"]
        admin = any(s in fixed for s in admin_types)
        if admin:
            deans_df.at[index, "FixedPosition"] = "Administration"
    return deans_df

def process_matches(deans_df):
    for index, row in deans_df.iterrows():
        fixed = row["FixedPosition"]
        if "Subinstitution Match" in fixed:
            classification = assign_position_grouping_subinst(row["Position"])
            deans_df.at[index, "FixedPosition"] = classification
    return deans_df

def cleanup(deans_df):
    cleanup_list=  ["Faculty", "Administrator", "Students", "Administrative", "Enrollment", "Finance", "Foundation", "Student", "Academic Affair", "Programs", "Alumni", "Relations", "Conservatory", "Division", "Professions", "Practice", "Educational", "Office", "Management"]
    for index, row in deans_df.iterrows():
        position = row["Position"].title()
        altered_position = remove_parentheses(position).strip()
        dean_index= altered_position.rfind("Dean") + 4
        if not pd.isna(row["SubInstitution"]):
            subinstitution = row["SubInstitution"].title()
        else:
            subinstitution = ""
        pos_to_remove = any(s in position for s in cleanup_list)
        sub_to_remove = any(s in subinstitution for s in cleanup_list)
        if dean_index != len(altered_position) and (pos_to_remove or sub_to_remove) and row["FixedPosition"] != "Administration":
            deans_df.at[index, "FixedPosition"] = "Administration"
            print(position, "        ", subinstitution)
    return deans_df
        

def expand_grouping(deans_df, subinstitution_df):
    subinstitution_dict = subinstitution_df.set_index(['Institution', 'SubInstitution'])['Category'].to_dict()
    grad_keys = ["graduate program", "graduate school", "graduate and", "advanced study", "graduate education", "graduate studies"]
    grad_key_title = [s.title() for s in grad_keys]
    for index, row in deans_df.iterrows():
        key = (row["Institution"], row["SubInstitution"])
        position = row["Position"].title()
        classification = row["FixedPosition"]
        if not pd.isna(row["SubInstitution"]):
            subinstitution = row["SubInstitution"].title()
        else:
            subinstitution = ""

        if "Graduate" in classification and subinstitution != "":
            deans_df.at[index, "FixedPosition"] = subinstitution_dict[key]
            if "Graduate" not in subinstitution_dict[key]:
                deans_df.at[index, "FixedPosition"] += ", Graduate"

        # if "Administration" not in classification and "Graduate" not in classification and "Graduate" in position:
        #     deans_df.at[index, "FixedPosition"] += ", Graduate"
        #     print(deans_df.at[index, "FixedPosition"])
        if "Satellite Campus" in subinstitution:
            deans_df.at[index, "FixedPosition"] += ", Satellite Campus"
        if "Librar" in row["Position"] and "Information" not in key:
            deans_df.at[index, "FixedPosition"] = "Administration"
        if "Continu" in row["Position"]:
            deans_df.at[index, "FixedPosition"] = "Continued Studies"
    return deans_df


def classify_other(deans_df, subinstitution_df):
    subinstitution_dict = subinstitution_df.set_index(['Institution', 'SubInstitution'])['Category'].to_dict()
    for index, row in deans_df.iterrows():
        if row["FixedPosition"] == "Other":
            continue
        


In [10]:
full_df = pd.read_csv(full_path)
subinstitution_df = pd.read_csv(grouped_subinstitution_path)

In [11]:
raw_deans = extract_deans(full_df)
cleaned_deans = assign_deans_subinstitution(raw_deans, subinstitution_df)
cleaned_deans = process_matches(cleaned_deans)
cleaned_deans = assign_remaining(cleaned_deans, subinstitution_df)
cleaned_deans = assign_none(cleaned_deans)
cleaned_deans = assign_administration(cleaned_deans)
cleaned_deans = cleanup(cleaned_deans)
cleaned_deans = expand_grouping(cleaned_deans, subinstitution_df)


Dean, Student Affairs          
Dean Of Students          
Dean Of Students          
Dean Of Students          
Dean Of Students          
Dean Of Students          
Dean Of Students          
Dean Of New Students          
Dean Of Students          
Dean Of Students          
Associate Dean Of Students, Residence Life          
Assistant Dean Of Students, Multicultural Affairs          
Dean, Student Affairs          
Dean Of Students          
Associate Dean, Enrollment Communications          
Dean Of Students          
Dean Of Students          
Associate Dean Of Students          
Dean Of Students          
Dean, Student Affairs          
Dean Of Students          
Dean Of Students          
Dean Of Students          
Associate Dean, Graduate Business Programs          
Dean Of Students          
Associate Dean, Graduate Programs          The Wallace E. Carroll School Of Management
Associate Dean, Graduate Programs          William F Connell School Of Nursing
Dean Of Students    

In [12]:
# arts and science & fine arts being categorized together but shouldnt
#administration cannot combine with others
for index, row in cleaned_deans.iterrows():
    category = row["FixedPosition"]
    if pd.isna(row["SubInstitution"]):
        subinstitution = ""
    else:
        subinstitution = row["SubInstitution"].title()
    institution = row["Institution"].title()
    word_list = list(subinstitution.split())
    art_count = word_list.count("Arts")
    engineering_count = word_list.count("Engineering")
    if "Arts," in subinstitution and "Arts And Sciences" not in category and "Liberal Arts" not in category and "Fine Arts" not in category:
        cleaned_deans.at[index, "FixedPosition"] += ", Fine Arts"
    if "And Science" in subinstitution and "Arts and Sciences" not in category and "Natural Sciences" not in category:
        cleaned_deans.at[index, "FixedPosition"] += ", Natural Sciences"
    if "Liberal Arts" in category and "Arts and Sciences" in category:
        cleaned_deans.at[index, "FixedPosition"] = category.replace("Liberal Arts", "").strip(", ")
    if "Software Engineering" in subinstitution and "Engineering" in category and engineering_count == 1:
        cleaned_deans.at[index, "FixedPosition"] = category.replace("Engineering", "").strip(", ")
    if "Hospitality" in subinstitution and "Hospital" in category:
        cleaned_deans.at[index, "FixedPosition"] = category.replace("Hospital,", "").strip(", ")
    if "Foundation School" in subinstitution:
        cleaned_deans.at[index, "FixedPosition"] = category.replace("Foundation", "").strip(", ")

In [13]:
def remove_consecutive_commas(string):
    return re.sub(r', ,+', ',', string)

for index, row in cleaned_deans.iterrows():
    catgeory = row["FixedPosition"]
    if ",," in category:
        cleaned_deans.at[index, category] = remove_consecutive_commas(category)

In [14]:
for index, row in cleaned_deans.iterrows():
    fixed = row["FixedPosition"].title()
    pos = row["Position"]
    if ("College Of" in pos or "School Of" in pos) and "Administration" in fixed:
        print(pos)

Dean, School Of Medicine And Medical Faculty
Dean, School Of Medicine And Medical Faculty
Executive Dean, College Of Food, Agricultural And Environmental Sciences
Dean, Division Of Biological Sciences And Pritzker School Of Medicine
Dean, School Of Health Professions
Dean, School Of Undergraduate Studies (Interim)
Associate Dean, Research And Graduate Programs, College Of Medicine


In [15]:
for index, row in cleaned_deans.iterrows():
    fixed = row["FixedPosition"].title()
    pos = row["Position"]
    sub = row["SubInstitution"]
    altered_pos = remove_parentheses(pos).strip()
    dean_index = altered_pos.rfind("Dean") + 4
    if dean_index == len(altered_pos) and ("Administration" in fixed or "Other" in fixed):
        print("Position: ", pos, "         " ,  "Subinstitution: ", sub, "          ", "Fixed: ", fixed)

Position:  Dean           Subinstitution:  University Library            Fixed:  Administration
Position:  Dean           Subinstitution:  University Library            Fixed:  Administration
Position:  Dean           Subinstitution:  Library and Information Access (Interim)            Fixed:  Administration
Position:  Dean           Subinstitution:  Undergraduate Studies            Fixed:  Administration
Position:  Dean           Subinstitution:  Library            Fixed:  Administration
Position:  Dean           Subinstitution:  Library Services            Fixed:  Administration
Position:  Dean           Subinstitution:  College of Freshman Studies            Fixed:  Administration
Position:  Dean           Subinstitution:  Faculty and Staff Relations            Fixed:  Administration
Position:  Dean           Subinstitution:  First Year Students            Fixed:  Administration
Position:  Dean           Subinstitution:  Admissions and Financial Aid            Fixed:  Administration

In [16]:
for index, row in cleaned_deans.iterrows():
    cleaned_deans.at[index, "FixedPosition"] = "Dean, " + row["FixedPosition"]
cleaned_deans = cleaned_deans.sort_values(by=["FixedPosition", "Institution"])
cleaned_deans.to_csv(dean_path, index = False)

In [17]:
category_counter = Counter()
for category_list in cleaned_deans["FixedPosition"]:
    categories = [cat.strip() for cat in category_list.split(",") if cat.strip()]
    category_counter.update(categories)

sorted_category = dict(sorted(category_counter.items(), key=lambda item: item[1], reverse=True))
print("\nCategory counts")
for category, number in sorted_category.items():
    print(f"{category}: {number}")


Category counts
Dean: 6338
Administration: 1842
Business: 538
Graduate: 413
Education: 405
Arts and Sciences: 326
Natural Sciences: 300
Engineering: 296
Health Science: 277
Fine Arts: 269
Law: 198
Medical: 188
Nursing: 184
Continued Studies: 138
Social Work: 133
Humanities: 120
Technology: 113
Social Sciences: 112
Liberal Arts: 104
Dean Of The College: 96
Communication: 89
Policy and Affairs: 86
Behavioral Science: 86
Pharmacy: 82
Agriculture: 80
Archictecure: 80
Environmental and Earth Science: 72
Dean of the College: 70
Satellite Campus: 68
Professional Studies: 65
Dentistry: 58
Information: 52
University College: 48
Honor College: 45
Computer Science: 43
Religious Studies: 37
Veterinary: 29
Journalism: 27
Student Affairs: 17
Physical Therapy and Nutrition: 15
Language and Culture: 14
DEI: 12
Hospitality: 11
Religion: 11
Optometry: 10
Family and Consumer Science: 9
Marine Science: 9
Error: 7
Criminal Justice: 6
Policy And Affairs: 4
Pathology: 2
Hospital: 2
Athletics: 1
Audiology: 1